<a href="https://colab.research.google.com/github/monoramasn/Speech_fairness/blob/main/dro_sd_erm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q git+https://github.com/huggingface/transformers.git@main git+https://github.com/huggingface/peft.git
!pip install -U accelerate
!pip install evaluate
!pip install jiwer
!pip install --upgrade transformers bitsandbytes datasets torch torchvision torchaudio

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
from datasets import load_dataset, DatasetDict
language="Estonian"
voxpopuli = DatasetDict()

voxpopuli["train"] = load_dataset("facebook/voxpopuli", "et", split=['train'])[0]
#voxpopuli["validation"] = load_dataset("facebook/voxpopuli", "et", split=['validation'])[0]
voxpopuli["test"] = load_dataset("facebook/voxpopuli", "et", split=['test'])[0]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/datasets/load.py:1461: FutureWarning: The repository for facebook/voxpopuli contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/facebook/voxpopuli
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  wa

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

/usr/local/lib/python3.10/dist-packages/datasets/load.py:1461: FutureWarning: The repository for facebook/voxpopuli contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/facebook/voxpopuli
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(


In [ ]:
# Sampling

#voxpopuli["train"] = voxpopuli["train"].select(range(800))
#voxpopuli["test"] = voxpopuli["test"].select(range(300))

voxpopuli_data = voxpopuli.remove_columns(['audio_id', 'language', 'raw_text', 'speaker_id', 'is_gold_transcript', 'accent'])

In [ ]:
from transformers import WhisperFeatureExtractor

model_name_or_path = "openai/whisper-small"
task = "transcribe"

feature_extractor = WhisperFeatureExtractor.from_pretrained(model_name_or_path)

preprocessor_config.json:   0%|          | 0.00/185k [00:00<?, ?B/s]

In [ ]:
from transformers import WhisperTokenizer
from transformers import WhisperProcessor

tokenizer = WhisperTokenizer.from_pretrained(model_name_or_path, language=language, task=task)
processor = WhisperProcessor.from_pretrained(model_name_or_path, language=language, task=task)

tokenizer_config.json:   0%|          | 0.00/283k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/836k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.48M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/494k [00:00<?, ?B/s]

normalizer.json:   0%|          | 0.00/52.7k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/34.6k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.19k [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
from transformers import WhisperForConditionalGeneration
from peft import prepare_model_for_kbit_training

model = WhisperForConditionalGeneration.from_pretrained(model_name_or_path, load_in_8bit=True, device_map="auto")
model = prepare_model_for_kbit_training(model)
#model = prepare_model_for_int8_training(model, output_embedding_layer_name="proj_out")

config.json:   0%|          | 0.00/1.97k [00:00<?, ?B/s]

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


model.safetensors:   0%|          | 0.00/967M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/3.87k [00:00<?, ?B/s]

In [ ]:
from peft import LoraConfig, PeftModel, LoraModel, LoraConfig, get_peft_model

config = LoraConfig(r=32, lora_alpha=64, target_modules=["q_proj", "v_proj"], lora_dropout=0.05, bias="none")

model = get_peft_model(model, config)
model.print_trainable_parameters()

trainable params: 3,538,944 || all params: 245,273,856 || trainable%: 1.442854145857274


In [ ]:
def activate_last_n_layers(model, n_active_layers):
    total_layers = model.config.num_hidden_layers
    layer_to_start_unfreeze = max(0, total_layers - n_active_layers)

    for name, param in model.named_parameters():
        if 'encoder.layers' in name:
            parts = name.split('.')
            layer_index_part = [part for part in parts if part.isdigit()]
            if layer_index_part:
                layer_index = int(layer_index_part[0])
                if param.dtype.is_floating_point or param.dtype.is_complex:
                    param.requires_grad = (layer_index >= layer_to_start_unfreeze)
            else:
                # Skip setting requires_grad if no digit part is found or if it's not a floating point/complex tensor
                continue
        else:
            # Check parameter data type before setting requires_grad
            if param.dtype.is_floating_point or param.dtype.is_complex:
                param.requires_grad = True


In [ ]:
def vary_layers_and_reapply_lora(model, config):
    for n in range(0, model.config.num_hidden_layers + 1):
        print(f"Activating the last {n} layers for training and reapplying LoRA.")
        activate_last_n_layers(model, n)

        # Reapply LoRA settings - assuming this also handles setting the newly active layers correctly
        model = get_peft_model(model, config)

In [ ]:
def vary_layers_and_reapply_lora(model, n_active_layers, config):
    # Activate the specified number of layers
    activate_last_n_layers(model, n_active_layers)

    # Reapply LoRA settings, potentially modifying the model according to the new active layer setup
    # Assuming get_peft_model correctly applies LoRA based on the updated active layers and provided config
    return get_peft_model(model, config)

In [ ]:
import torch
#device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
#model.to(device)

In [ ]:
# Data Preparation Function
def prepare_dataset(batch):
    batch_input_features = []
    batch_labels = []
    batch_genders = []

    for idx in range(len(batch["audio"])):
        audio_features = feature_extractor(batch["audio"][idx]["array"], sampling_rate=batch["audio"][idx]["sampling_rate"]).input_features
        batch_input_features.extend(audio_features)

        label = tokenizer(batch["normalized_text"][idx]).input_ids
        batch_labels.append(label)

        # Assuming 'gender' field exists and contains strings like "male" or "female"
        gender = 1 if batch["gender"][idx] == "male" else 2  # Map "male" to 1, "female" to 2
        batch_genders.append(gender)

    return {"input_features": batch_input_features, "labels": batch_labels, "groups": batch_genders}

In [ ]:
voxpopuli_data = voxpopuli.map(prepare_dataset, remove_columns=voxpopuli.column_names["train"], batched=True)

Map:   0%|          | 0/834 [00:00<?, ? examples/s]

Map:   0%|          | 0/51 [00:00<?, ? examples/s]

In [ ]:
voxpopuli_data

DatasetDict({
    train: Dataset({
        features: ['input_features', 'labels', 'groups'],
        num_rows: 834
    })
    test: Dataset({
        features: ['input_features', 'labels', 'groups'],
        num_rows: 51
    })
})

In [ ]:
import torch

from dataclasses import dataclass
from typing import Any, Dict, List, Union

@dataclass
class DataCollatorSpeechSeq2SeqWithPadding:
    def __init__(self, processor):
        self.processor = processor

    def __call__(self, features):
        input_features = [{"input_features": feature["input_features"]} for feature in features]
        batch = self.processor.feature_extractor.pad(input_features, return_tensors="pt")

        label_features = [{"input_ids": feature["labels"]} for feature in features]
        labels_batch = self.processor.tokenizer.pad(label_features, return_tensors="pt")
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)
        if labels.size(1) > 0 and (labels[:, 0] == self.processor.tokenizer.bos_token_id).all().cpu().item():
            labels = labels[:, 1:]
        batch["labels"] = labels

        # Directly use 'groups' from features, assuming they are already numeric
        batch["groups"] = torch.tensor([feature["groups"] for feature in features], dtype=torch.long)

        return batch

In [ ]:
from torch.utils.data import DataLoader

data_collator = DataCollatorSpeechSeq2SeqWithPadding(processor=processor)

In [ ]:
from torch.utils.data import DataLoader

# Assuming voxpopuli_sl is a dictionary-like object containing your processed training and testing datasets
batch_size = 4  # or any other batch size fitting your GPU memory

train_dataloader = DataLoader(voxpopuli_data["train"], batch_size=batch_size, shuffle=True, collate_fn=data_collator)
#val_dataloader = DataLoader(voxpopuli_data["validation"], batch_size=batch_size, shuffle=True, collate_fn=data_collator)

In [ ]:
def compute_erm_loss(outputs, labels, erm_loss_func):
    return erm_loss_func(outputs.logits.view(-1, outputs.logits.size(-1)), labels.view(-1))

def compute_group_dro_loss(outputs, labels, groups, erm_loss_func):
    unique_groups = groups.unique()
    group_losses = []
    for group in unique_groups:
        group_indices = (groups == group)
        group_loss = erm_loss_func(outputs.logits[group_indices].view(-1, outputs.logits.size(-1)), labels[group_indices].view(-1))
        group_losses.append(group_loss)
    return max(group_losses) if group_losses else torch.tensor(0.0, device=outputs.logits.device)

def compute_spectral_norm(model):
    # Only compute norms for floating point tensors
    return sum(torch.linalg.norm(p).item() for p in model.parameters() if p.dtype.is_floating_point)


In [ ]:
class CustomLoss(torch.nn.Module):
    def __init__(self, model, erm_loss_func=torch.nn.CrossEntropyLoss(), lambda_erm=1.0, lambda_dro=1.0, lambda_spectral=0.004):
        super().__init__()
        self.model = model
        self.erm_loss_func = erm_loss_func
        self.lambda_erm = lambda_erm
        self.lambda_dro = lambda_dro
        self.lambda_spectral = lambda_spectral

    def forward(self, outputs, labels, groups):
        # ERM component
        loss_erm = compute_erm_loss(outputs, labels, self.erm_loss_func)

        # Group DRO component
        loss_dro = compute_group_dro_loss(outputs, labels, groups, self.erm_loss_func)

        # Spectral Decoupling component
        spectral_norm = compute_spectral_norm(self.model)

        # Combine the losses
        total_loss = (self.lambda_erm * loss_erm) + (self.lambda_dro * loss_dro) + (self.lambda_spectral * spectral_norm)
        return total_loss

In [ ]:
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
import evaluate

metric = evaluate.load("wer")

In [ ]:
from torch.optim import AdamW

# Assuming model, CustomLoss, train_dataloader, device, and config are defined
optimizer = AdamW(model.parameters(), lr=5e-4)
custom_loss_func = CustomLoss(model=model, lambda_erm=1.0, lambda_dro=1.0, lambda_spectral=0.004).to(device)
number_of_epochs = 5
total_layers = 12

for n_active_layers in range(0, total_layers + 1):  # Loop over number of active layers
    print(f"\nTraining with the last {n_active_layers} layers active.")
    vary_layers_and_reapply_lora(model, n_active_layers, config)  # Adjust function to pass n_active_layers and config

    for epoch in range(number_of_epochs):
        model.train()
        total_loss, total_male_loss, total_female_loss = 0, 0, 0
        male_count, female_count = 0, 0

        for batch in train_dataloader:
            optimizer.zero_grad()

            input_features = batch['input_features'].to(device)
            labels = batch['labels'].to(device)
            groups = batch['groups'].to(device) if 'groups' in batch else None

            outputs = model(input_features=input_features, labels=labels, return_dict=True)

            # General loss calculation
            loss = custom_loss_func(outputs, labels, groups)
            total_loss += loss.item()
            loss.backward()
            optimizer.step()

            # Separate loss calculations for male and female, if groups information exists
            if groups is not None:
                male_mask = (groups == 1)
                female_mask = (groups == 2)

                if male_mask.any():
                    male_loss = custom_loss_func(outputs, labels, groups * male_mask.long())
                    total_male_loss += male_loss.item()
                    male_count += male_mask.sum().item()

                if female_mask.any():
                    female_loss = custom_loss_func(outputs, labels, groups * female_mask.long())
                    total_female_loss += female_loss.item()
                    female_count += female_mask.sum().item()

        avg_loss = total_loss / len(train_dataloader)
        avg_male_loss = total_male_loss / male_count if male_count > 0 else 0
        avg_female_loss = total_female_loss / female_count if female_count > 0 else 0
        print(f"Config {n_active_layers}, Epoch {epoch+1}/{number_of_epochs}, Average Loss: {avg_loss:.2f}")
        if male_count > 0:
            print(f"Average Male Loss: {avg_male_loss:.2f}")
        if female_count > 0:
            print(f"Average Female Loss: {avg_female_loss:.2f}")


Training with the last 0 layers active.


/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:460: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:90: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/bitsandbytes/autograd/_functions.py:322: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")
`use_cache = True` is incompatible with gradient checkpointing. Setting `use_cache = False`...


Config 0, Epoch 1/5, Average Loss: 30.96
Average Male Loss: 14.75
Average Female Loss: 14.81
Config 0, Epoch 2/5, Average Loss: 31.05
Average Male Loss: 14.64
Average Female Loss: 14.58
Config 0, Epoch 3/5, Average Loss: 31.01
Average Male Loss: 14.55
Average Female Loss: 14.55
Config 0, Epoch 4/5, Average Loss: 31.00
Average Male Loss: 14.69
Average Female Loss: 14.69
Config 0, Epoch 5/5, Average Loss: 31.00
Average Male Loss: 14.69
Average Female Loss: 14.55

Training with the last 1 layers active.
Config 1, Epoch 1/5, Average Loss: 31.05
Average Male Loss: 14.79
Average Female Loss: 14.14
Config 1, Epoch 2/5, Average Loss: 30.96
Average Male Loss: 14.67
Average Female Loss: 14.16
Config 1, Epoch 3/5, Average Loss: 30.99
Average Male Loss: 14.45
Average Female Loss: 14.18
Config 1, Epoch 4/5, Average Loss: 31.04
Average Male Loss: 14.71
Average Female Loss: 14.57
Config 1, Epoch 5/5, Average Loss: 31.02
Average Male Loss: 14.93
Average Female Loss: 14.17

Training with the last 2 lay

In [ ]:
import torch
from torch.utils.data import DataLoader
import evaluate

# Initialize the WER metric
metric = evaluate.load("wer")

# Create the DataLoader for the evaluation dataset
eval_dataloader = DataLoader(voxpopuli_data["test"], batch_size=8, collate_fn=data_collator)

total_layers = model.config.num_hidden_layers  # Get the total number of transformer layers

# Iterate over the number of layers to activate from 1 to total number of layers
for n_active_layers in range(0, total_layers + 1):
    # Adjust the model for the current number of active layers
    model.config.num_hidden_layers = n_active_layers

    # Ensure the model is in evaluation mode
    model.eval()

    # Initialize accumulators for WER calculations
    total_wer, total_male_wer, total_female_wer = 0, 0, 0
    male_count, female_count, total_count = 0, 0, 0

    # Disable gradient calculations for evaluation
    with torch.no_grad():
        for batch in eval_dataloader:
            input_features = batch['input_features'].to(device)
            labels = batch['labels'].to(device)
            groups = batch['groups'].to(device)

            # Forward pass
            outputs = model(input_features=input_features, labels=labels, return_dict=True)
            predictions = outputs.logits.argmax(dim=-1)

            # Decode predictions and references
            for idx, (prediction_ids, label_ids, group) in enumerate(zip(predictions, labels, groups)):
                hypothesis_text = tokenizer.decode(prediction_ids, skip_special_tokens=True)
                reference_text = tokenizer.decode(label_ids, skip_special_tokens=True)

                # Calculate WER for this sample
                wer = metric.compute(predictions=[hypothesis_text], references=[reference_text])

                # Accumulate WER and count for averages
                total_wer += wer
                total_count += 1

                # Accumulate male and female WER separately
                if group.item() == 1:  # Male
                    total_male_wer += wer
                    male_count += 1
                elif group.item() == 2:  # Female
                    total_female_wer += wer
                    female_count += 1



Configuration with the last 0 active layers:
Combined Average WER: 0.69
Male Average WER: 0.67
Female Average WER: 0.70
Configuration with the last 1 active layers:
Combined Average WER: 0.69
Male Average WER: 0.67
Female Average WER: 0.70
Configuration with the last 2 active layers:
Combined Average WER: 0.69
Male Average WER: 0.67
Female Average WER: 0.70
Configuration with the last 3 active layers:
Combined Average WER: 0.69
Male Average WER: 0.67
Female Average WER: 0.70
Configuration with the last 4 active layers:
Combined Average WER: 0.69
Male Average WER: 0.67
Female Average WER: 0.70
Configuration with the last 5 active layers:
Combined Average WER: 0.69
Male Average WER: 0.67
Female Average WER: 0.70
Configuration with the last 6 active layers:
Combined Average WER: 0.69
Male Average WER: 0.67
Female Average WER: 0.70
Configuration with the last 7 active layers:
Combined Average WER: 0.69
Male Average WER: 0.67
Female Average WER: 0.70
Configuration with the last 8 active lay

erm

In [ ]:
from torch.optim import AdamW

# Assuming model, CustomLoss, train_dataloader, device, and config are defined
optimizer = AdamW(model.parameters(), lr=5e-4)
custom_loss_func = CustomLoss(model=model, lambda_erm=1.0, lambda_dro=0, lambda_spectral=0).to(device)
number_of_epochs = 5
total_layers = 12

for n_active_layers in range(0, total_layers + 1):  # Loop over number of active layers
    print(f"\nTraining with the last {n_active_layers} layers active.")
    vary_layers_and_reapply_lora(model, n_active_layers, config)  # Adjust function to pass n_active_layers and config

    for epoch in range(number_of_epochs):
        model.train()
        total_loss, total_male_loss, total_female_loss = 0, 0, 0
        male_count, female_count = 0, 0

        for batch in train_dataloader:
            optimizer.zero_grad()

            input_features = batch['input_features'].to(device)
            labels = batch['labels'].to(device)
            groups = batch['groups'].to(device) if 'groups' in batch else None

            outputs = model(input_features=input_features, labels=labels, return_dict=True)

            # General loss calculation
            loss = custom_loss_func(outputs, labels, groups)
            total_loss += loss.item()
            loss.backward()
            optimizer.step()

            # Separate loss calculations for male and female, if groups information exists
            if groups is not None:
                male_mask = (groups == 1)
                female_mask = (groups == 2)

                if male_mask.any():
                    male_loss = custom_loss_func(outputs, labels, groups * male_mask.long())
                    total_male_loss += male_loss.item()
                    male_count += male_mask.sum().item()

                if female_mask.any():
                    female_loss = custom_loss_func(outputs, labels, groups * female_mask.long())
                    total_female_loss += female_loss.item()
                    female_count += female_mask.sum().item()

        avg_loss = total_loss / len(train_dataloader)
        avg_male_loss = total_male_loss / male_count if male_count > 0 else 0
        avg_female_loss = total_female_loss / female_count if female_count > 0 else 0
        print(f"Config {n_active_layers}, Epoch {epoch+1}/{number_of_epochs}, Average Loss: {avg_loss:.2f}")
        if male_count > 0:
            print(f"Average Male Loss: {avg_male_loss:.2f}")
        if female_count > 0:
            print(f"Average Female Loss: {avg_female_loss:.2f}")


In [ ]:
import torch
from torch.utils.data import DataLoader
import evaluate

# Initialize the WER metric
metric = evaluate.load("wer")

# Create the DataLoader for the evaluation dataset
eval_dataloader = DataLoader(voxpopuli_data["test"], batch_size=8, collate_fn=data_collator)

total_layers = model.config.num_hidden_layers  # Get the total number of transformer layers

# Iterate over the number of layers to activate from 1 to total number of layers
for n_active_layers in range(0, total_layers + 1):
    # Adjust the model for the current number of active layers
    model.config.num_hidden_layers = n_active_layers

    # Ensure the model is in evaluation mode
    model.eval()

    # Initialize accumulators for WER calculations
    total_wer, total_male_wer, total_female_wer = 0, 0, 0
    male_count, female_count, total_count = 0, 0, 0

    # Disable gradient calculations for evaluation
    with torch.no_grad():
        for batch in eval_dataloader:
            input_features = batch['input_features'].to(device)
            labels = batch['labels'].to(device)
            groups = batch['groups'].to(device)

            # Forward pass
            outputs = model(input_features=input_features, labels=labels, return_dict=True)
            predictions = outputs.logits.argmax(dim=-1)

            # Decode predictions and references
            for idx, (prediction_ids, label_ids, group) in enumerate(zip(predictions, labels, groups)):
                hypothesis_text = tokenizer.decode(prediction_ids, skip_special_tokens=True)
                reference_text = tokenizer.decode(label_ids, skip_special_tokens=True)

                # Calculate WER for this sample
                wer = metric.compute(predictions=[hypothesis_text], references=[reference_text])

                # Accumulate WER and count for averages
                total_wer += wer
                total_count += 1

                # Accumulate male and female WER separately
                if group.item() == 1:  # Male
                    total_male_wer += wer
                    male_count += 1
                elif group.item() == 2:  # Female
                    total_female_wer += wer
                    female_count += 1

        # Calculate and print average WER for the current layer configuration
        avg_wer = total_wer / total_count if total_count > 0 else 0
        avg_male_wer = total_male_wer / male_count if male_count > 0 else 0
        avg_female_wer = total_female_wer / female_count if female_count > 0 else 0
        print(f"Configuration with the last {n_active_layers} active layers:")
        print(f"Combined Average WER: {avg_wer:.2f}")
        print(f"Male Average WER: {avg_male_wer:.2f}")
        print(f"Female Average WER: {avg_female_wer:.2f}")

dro

In [ ]:
from torch.optim import AdamW

# Assuming model, CustomLoss, train_dataloader, device, and config are defined
optimizer = AdamW(model.parameters(), lr=5e-5)
custom_loss_func = CustomLoss(model=model, lambda_erm=0, lambda_dro=1.0, lambda_spectral=0).to(device)
number_of_epochs = 5
total_layers = 12
for n_active_layers in range(0, total_layers + 1):  # Loop over number of active layers
    print(f"\nTraining with the last {n_active_layers} layers active.")
    vary_layers_and_reapply_lora(model, n_active_layers, config)  # Adjust function to pass n_active_layers and config

    for epoch in range(number_of_epochs):
        model.train()
        total_loss, total_male_loss, total_female_loss = 0, 0, 0
        male_count, female_count = 0, 0

        for batch in train_dataloader:
            optimizer.zero_grad()

            input_features = batch['input_features'].to(device)
            labels = batch['labels'].to(device)
            groups = batch['groups'].to(device) if 'groups' in batch else None

            outputs = model(input_features=input_features, labels=labels, return_dict=True)

            # General loss calculation
            loss = custom_loss_func(outputs, labels, groups)
            total_loss += loss.item()
            loss.backward()
            optimizer.step()

            # Separate loss calculations for male and female, if groups information exists
            if groups is not None:
                male_mask = (groups == 1)
                female_mask = (groups == 2)

                if male_mask.any():
                    male_loss = custom_loss_func(outputs, labels, groups * male_mask.long())
                    total_male_loss += male_loss.item()
                    male_count += male_mask.sum().item()

                if female_mask.any():
                    female_loss = custom_loss_func(outputs, labels, groups * female_mask.long())
                    total_female_loss += female_loss.item()
                    female_count += female_mask.sum().item()

        avg_loss = total_loss / len(train_dataloader)
        avg_male_loss = total_male_loss / male_count if male_count > 0 else 0
        avg_female_loss = total_female_loss / female_count if female_count > 0 else 0
        print(f"Config {n_active_layers}, Epoch {epoch+1}/{number_of_epochs}, Average Loss: {avg_loss:.2f}")
        if male_count > 0:
            print(f"Average Male Loss: {avg_male_loss:.2f}")
        if female_count > 0:
            print(f"Average Female Loss: {avg_female_loss:.2f}")


In [ ]:
import torch
from torch.utils.data import DataLoader
import evaluate

# Initialize the WER metric
metric = evaluate.load("wer")

# Create the DataLoader for the evaluation dataset
eval_dataloader = DataLoader(voxpopuli_data["test"], batch_size=8, collate_fn=data_collator)

total_layers = model.config.num_hidden_layers  # Get the total number of transformer layers

# Iterate over the number of layers to activate from 1 to total number of layers
for n_active_layers in range(0, total_layers + 1):
    # Adjust the model for the current number of active layers
    model.config.num_hidden_layers = n_active_layers

    # Ensure the model is in evaluation mode
    model.eval()

    # Initialize accumulators for WER calculations
    total_wer, total_male_wer, total_female_wer = 0, 0, 0
    male_count, female_count, total_count = 0, 0, 0

    # Disable gradient calculations for evaluation
    with torch.no_grad():
        for batch in eval_dataloader:
            input_features = batch['input_features'].to(device)
            labels = batch['labels'].to(device)
            groups = batch['groups'].to(device)

            # Forward pass
            outputs = model(input_features=input_features, labels=labels, return_dict=True)
            predictions = outputs.logits.argmax(dim=-1)

            # Decode predictions and references
            for idx, (prediction_ids, label_ids, group) in enumerate(zip(predictions, labels, groups)):
                hypothesis_text = tokenizer.decode(prediction_ids, skip_special_tokens=True)
                reference_text = tokenizer.decode(label_ids, skip_special_tokens=True)

                # Calculate WER for this sample
                wer = metric.compute(predictions=[hypothesis_text], references=[reference_text])

                # Accumulate WER and count for averages
                total_wer += wer
                total_count += 1

                # Accumulate male and female WER separately
                if group.item() == 1:  # Male
                    total_male_wer += wer
                    male_count += 1
                elif group.item() == 2:  # Female
                    total_female_wer += wer
                    female_count += 1

        # Calculate and print average WER for the current layer configuration
        avg_wer = total_wer / total_count if total_count > 0 else 0
        avg_male_wer = total_male_wer / male_count if male_count > 0 else 0
        avg_female_wer = total_female_wer / female_count if female_count > 0 else 0
        print(f"Configuration with the last {n_active_layers} active layers:")
        print(f"Combined Average WER: {avg_wer:.2f}")
        print(f"Male Average WER: {avg_male_wer:.2f}")
        print(f"Female Average WER: {avg_female_wer:.2f}")

spectral-decoupeling

In [ ]:
from torch.optim import AdamW

# Assuming model, CustomLoss, train_dataloader, device, and config are defined
optimizer = AdamW(model.parameters(), lr=5e-5)
custom_loss_func = CustomLoss(model=model, lambda_erm=0, lambda_dro=0, lambda_spectral=0.004).to(device)
number_of_epochs = 5
total_layers = 12

for n_active_layers in range(0, total_layers + 1):  # Loop over number of active layers
    print(f"\nTraining with the last {n_active_layers} layers active.")
    vary_layers_and_reapply_lora(model, n_active_layers, config)  # Adjust function to pass n_active_layers and config

    for epoch in range(number_of_epochs):
        model.train()
        total_loss, total_male_loss, total_female_loss = 0, 0, 0
        male_count, female_count = 0, 0

        for batch in train_dataloader:
            optimizer.zero_grad()

            input_features = batch['input_features'].to(device)
            labels = batch['labels'].to(device)
            groups = batch['groups'].to(device) if 'groups' in batch else None

            outputs = model(input_features=input_features, labels=labels, return_dict=True)

            # General loss calculation
            loss = custom_loss_func(outputs, labels, groups)
            total_loss += loss.item()
            loss.backward()
            optimizer.step()

            # Separate loss calculations for male and female, if groups information exists
            if groups is not None:
                male_mask = (groups == 1)
                female_mask = (groups == 2)

                if male_mask.any():
                    male_loss = custom_loss_func(outputs, labels, groups * male_mask.long())
                    total_male_loss += male_loss.item()
                    male_count += male_mask.sum().item()

                if female_mask.any():
                    female_loss = custom_loss_func(outputs, labels, groups * female_mask.long())
                    total_female_loss += female_loss.item()
                    female_count += female_mask.sum().item()

        avg_loss = total_loss / len(train_dataloader)
        avg_male_loss = total_male_loss / male_count if male_count > 0 else 0
        avg_female_loss = total_female_loss / female_count if female_count > 0 else 0
        print(f"Config {n_active_layers}, Epoch {epoch+1}/{number_of_epochs}, Average Loss: {avg_loss:.2f}")
        if male_count > 0:
            print(f"Average Male Loss: {avg_male_loss:.2f}")
        if female_count > 0:
            print(f"Average Female Loss: {avg_female_loss:.2f}")


In [ ]:
import torch
from torch.utils.data import DataLoader
import evaluate

# Initialize the WER metric
metric = evaluate.load("wer")

# Create the DataLoader for the evaluation dataset
eval_dataloader = DataLoader(voxpopuli_data["test"], batch_size=8, collate_fn=data_collator)

total_layers = model.config.num_hidden_layers  # Get the total number of transformer layers

# Iterate over the number of layers to activate from 1 to total number of layers
for n_active_layers in range(0, total_layers + 1):
    # Adjust the model for the current number of active layers
    model.config.num_hidden_layers = n_active_layers

    # Ensure the model is in evaluation mode
    model.eval()

    # Initialize accumulators for WER calculations
    total_wer, total_male_wer, total_female_wer = 0, 0, 0
    male_count, female_count, total_count = 0, 0, 0

    # Disable gradient calculations for evaluation
    with torch.no_grad():
        for batch in eval_dataloader:
            input_features = batch['input_features'].to(device)
            labels = batch['labels'].to(device)
            groups = batch['groups'].to(device)

            # Forward pass
            outputs = model(input_features=input_features, labels=labels, return_dict=True)
            predictions = outputs.logits.argmax(dim=-1)

            # Decode predictions and references
            for idx, (prediction_ids, label_ids, group) in enumerate(zip(predictions, labels, groups)):
                hypothesis_text = tokenizer.decode(prediction_ids, skip_special_tokens=True)
                reference_text = tokenizer.decode(label_ids, skip_special_tokens=True)

                # Calculate WER for this sample
                wer = metric.compute(predictions=[hypothesis_text], references=[reference_text])

                # Accumulate WER and count for averages
                total_wer += wer
                total_count += 1

                # Accumulate male and female WER separately
                if group.item() == 1:  # Male
                    total_male_wer += wer
                    male_count += 1
                elif group.item() == 2:  # Female
                    total_female_wer += wer
                    female_count += 1

        # Calculate and print average WER for the current layer configuration
        avg_wer = total_wer / total_count if total_count > 0 else 0
        avg_male_wer = total_male_wer / male_count if male_count > 0 else 0
        avg_female_wer = total_female_wer / female_count if female_count > 0 else 0
        print(f"Configuration with the last {n_active_layers} active layers:")
        print(f"Combined Average WER: {avg_wer:.2f}")
        print(f"Male Average WER: {avg_male_wer:.2f}")
        print(f"Female Average WER: {avg_female_wer:.2f}")